이번 챕터에서는 더 많은 데이터 문자 단위 RNN을 구현합니다.

# 1. 문자 단위 RNN(Char RNN)

---

우선 필요한 도구들을 임포트합니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

다음과 같이 임의의 샘플을 만듭니다.

## 1. 훈련 데이터 전처리하기

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

문자 집합을 생성하고, 각 문자에 고유한 정수를 부여합니다.

In [3]:
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩

In [4]:
print(char_dic) # 공백도 여기서는 하나의 원소

{'t': 0, 'i': 1, 'p': 2, 'b': 3, 'd': 4, 'm': 5, ',': 6, 'n': 7, 'g': 8, 'r': 9, 'k': 10, '.': 11, ' ': 12, 'o': 13, 's': 14, 'a': 15, 'f': 16, "'": 17, 'e': 18, 'l': 19, 'h': 20, 'y': 21, 'w': 22, 'u': 23, 'c': 24}


각 문자에 정수가 부여되었으며, 총 25개의 문자가 존재합니다. 문자 집합의 크기를 확인해봅시다.

In [5]:
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 25


문자 집합의 크기는 25이며, 입력을 원-핫 벡터로 사용할 것이므로 이는 매 시점마다 들어갈 입력의 크기이기도 합니다. 이제 하이퍼파라미터를 설정합니다. hidden_size(은닉 상태의 크기)를 입력의 크기와 동일하게 줬는데, 이는 사용자의 선택으로 다른 값을 줘도 무방합니다.

그리고 sequence_length라는 변수를 선언했는데, 우리가 앞서 만든 샘플을 10개 단위로 끊어서 샘플을 만들 예정이기 때문입니다. 이는 뒤에서 더 자세히 보겠습니다.

In [6]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.1

다음은 임의로 지정한 sequence_length 값인 10의 단위로 샘플들을 잘라서 데이터를 만드는 모습을 보여줍니다.

즉 이 RNN model은 'abcdefghij'을 input으로 넣으면 'bcdefghij' 식으로 나오는 model인 것입니다. (편의상 sentence에 없는 알파벳을 사용)

In [7]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

총 170개의 샘플이 생성되었습니다. 그리고 각 샘플의 각 문자들은 고유한 정수로 인코딩이 된 상태입니다. 첫번째 샘플의 입력 데이터와 레이블 데이터를 출력해봅시다.

In [8]:
print(x_data[0]) # if you wan에 해당됨.
print(y_data[0]) # f you want에 해당됨.

[1, 16, 12, 21, 13, 23, 12, 22, 15, 7]
[16, 12, 21, 13, 23, 12, 22, 15, 7, 0]


한 칸씩 쉬프트 된 시퀀스가 정상적으로 출력되는 것을 볼 수 있습니다. 이제 입력 시퀀스에 대해서 원-핫 인코딩을 수행하고, 입력 데이터와 레이블 데이터를 텐서로 변환합니다.

In [10]:
x_one_hot = np.array([np.eye(dic_size)[x] for x in x_data]) # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

이제 훈련 데이터와 레이블 데이터의 크기를 확인해봅시다.

In [11]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


원-핫 인코딩 된 결과를 보기 위해서 첫번째 샘플만 출력해봅시다.

In [12]:
print(X[0])

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

레이블 데이터의 첫번째 샘플도 출력해봅시다.

In [13]:
print(Y[0])

tensor([16, 12, 21, 13, 23, 12, 22, 15,  7,  0])


위 레이블 시퀀스는 f you want에 해당됩니다. 이제 모델을 설계합니다.

# 2. 모델 구현하기

---

모델은 앞서 실습한 문자 단위 RNN 챕터와 거의 동일합니다. 다만 이번에는 은닉층을 두 개 쌓을 겁니다.

In [14]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [15]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.

nn.RNN() 안에 num_layers라는 인자를 사용합니다. 이는 은닉층을 몇 개 쌓을 것인지를 의미합니다. 모델 선언 시 layers라는 인자에 2를 전달하여 은닉층을 두 개 쌓습니다. 비용 함수와 옵티마이저를 선언합니다.

In [16]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

이제 모델에 입력을 넣어서 출력의 크기를 확인해봅시다.

In [17]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


(170, 10, 25)의 크기를 가지는데 각각 배치 차원, 시점(timesteps), 출력의 크기입니다. 나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산하게 되는데, 이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만듭니다.

In [18]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.

torch.Size([1700, 25])


차원이 (1700, 25)가 된 것을 볼 수 있습니다. 이제 레이블 데이터의 크기를 다시 복습해 봅시다.

In [19]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


레이블 데이터는 (170, 10)의 크기를 가지는데, 마찬가지로 나중에 정확도를 측정할 때는 이걸 펼쳐서 계산할 예정입니다. 이 경우 (1700)의 크기를 가지게 됩니다. 이제 옵티마이저와 손실 함수를 정의합니다.

In [20]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

seellgsscgsssyhssleeeshyygsyycsgclycsgcsscssssheshshhssegsyshhsseygshyychhssgcsshgslyseysyghsyggsscfygycgcsychsyeeylsscdecsyshsysysydysshhecgssychssecsshcegeyyshysghsdhcehesedsshs
                                                                                                                                                                                   
eeei  eii t    t a  a   a   t a t t    t  t   aa   e   e  to    o a       o   t e t a       oe  t  aa o   t   o a   t t e  to a e  t  t   a   t t t       a a   a t  o  t  a e a   
                                                                                                                                                                                   
m        d dd  d md dd dd d d d e   dd   md md dd dddd d  dd  md a dd dd     dd   d d dd    dd    d d    d d  e d  d  d a d  d md d dd dd d    d  d    dd edd  d d    d  dd d  d d 
piimi pi liiii iomeoiemiipi imiin i ie ui iipi iimiepi i  ioi pi iiiinii mmipei   e eoii iii e p  e 

g yoo want to build asship, don't trum up people to ether te lollect wood and won't tssign them tosks and dook, but rathe  teach them to long for the sndless immensity on the soas
g yo  want to luild asship, don't drum up peo le to ether te lollect wood and don't dssign them tosks and dook, but rathe  teach them to long for the sndless immensity on the soas
m yop want to luild asshap, don't drum up people to ether te lollect wood and don't dssign them tosks and dook, but rathe  teach them to long for the sndless immensity of the soas
m yop want to build a shap, don't drum up people to ether te lollect wood and don't dssign them tosks and dook, but rathe  teach them ta long for the sndless immensity of the eoas
m yop want to build a ship, don't drum up people to ether to lollect wopd and don't dssign them tasks and dohk, but rathe  toach them ta long for the sndless immensity of the soas
m yop want to build a ship, don't drum up people to ether to lollect wopd and don't dssign them task

처음에는 이상한 예측을 하지만 마지막 에포크에서는 꽤 정확한 문자을 생성하는 것을 볼 수 있습니다.